In [1]:
import gradio as gr

In [2]:
import os
from together import Together
api_key = os.getenv("together_api_key")
client = Together(api_key=api_key)


def get_results_from_model(input):

    prompts = f'''
    You are expert in generating steps and instructions for a mathematical image generator.
    The output should contain the following elements:
    * if the axis names in the image are necessary, give a dummy names in the context of mathematics
    * the range of the axes
    * labels corresponding to the important parts of the graph.

    ```Note: Make sure to be precise and short. Strictly return less than or equal to 150 words output```

    user input: {input}
    output: 

    '''

    client = Together(api_key=api_key)
    response = client.chat.completions.create(
       model="meta-llama/Llama-3-70b-chat-hf",
       messages=[{"role": "user", "content": prompts}],
    )

    # Extract the selected option from the response
    response = response.choices[0].message.content.strip()
    return response

In [3]:
# # Create the Gradio interface
# iface = gr.Interface(
#     fn=get_results_from_model,
#     inputs=gr.Textbox(placeholder="Enter a topic or question..."),
#     outputs="text",
#     title="LLaMA 3 Text Generation",
#     description="Generate text about a given topic using the LLaMA 3 model."
# )

# # Launch the Gradio app
# iface.launch()


In [4]:
import gradio as gr
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt

In [5]:

# Load GPT-2 model for text generation
# text_generator = pipeline('text-generation', model='gpt2')

# # Load Stable Diffusion model for image generation
image_generator = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
# image_generator.to("cuda")

def generate_text_and_image(input_text):
    text_output = get_results_from_model(input_text)
    image = image_generator(text_output).images[0]
    # image = plt.imread("/Users/gopimaguluri/Downloads/content 2/train/image_94.png")
    
    return text_output, image


# # Create Gradio interface
# iface = gr.Interface(
#     fn=generate_text_and_image,
#     inputs=gr.Textbox(label="Enter a prompt"),
#     outputs=[
#         gr.Textbox(label="Generated Text Instructions"),
#         gr.Image(label="Generated Image")
#     ],
#     title="Text to Image Generation",
#     description="Enter a prompt to generate text instructions and an image based on those instructions."
# )

# Launch the interface
# iface.launch()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/anaconda3/envs/texttoimage/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Create Gradio interface
with gr.Blocks(theme="default") as iface:
    gr.Markdown("# EduGraphix")
    gr.Markdown("What are you exploring today? Let us enhance your journey with visual aids!")
    
    with gr.Row():
        with gr.Column(scale=1):
            input_text = gr.Textbox(label="Enter a prompt", lines=5)
            # Small Submit Button below the input text box
            submit_button = gr.Button("Generate", elem_id="submit-button", size="small")
        with gr.Column(scale=1):
            text_output = gr.Textbox(label="Textual Description", lines=5)
    
    with gr.Row():
        with gr.Column(scale=1, min_width=0):
            gr.Markdown("")  # Empty column for spacing
        with gr.Column(scale=2):
            image_output = gr.Image(label="Generated Image")
        with gr.Column(scale=1, min_width=0):
            gr.Markdown("")  # Empty column for spacing
    
    # Set up the button action
    submit_button.click(generate_text_and_image, inputs=input_text, outputs=[text_output, image_output])

# Add custom CSS to style the button smaller
iface.css = """
#submit-button {
    padding: 5px 10px; /* Adjust padding for smaller size */
    font-size: 12px;   /* Smaller font size */
}
"""

# Launch the interface
iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Token indices sequence length is longer than the specified maximum sequence length for this model (194 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['deviations from the mean ) * y - axis : 0 to 0. 4 ( probability density ) ** labels :** * mean ( μ ) at x = 0 * standard deviation ( σ ) at x = 1 and x = - 1 * peak of the curve at x = 0, y = 0. 4 ( maximum probability density ) ** additional features :** * shade the area under the curve to represent 6 8. 2 7 % of the data within 1 standard deviation of the mean. * add a vertical line at x = 0 to highlight the mean.']


  0%|          | 0/50 [00:00<?, ?it/s]